In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder

In [ ]:
business_data = pd.read_csv('business_d.csv')
user_data = pd.read_csv('user_d.csv')
time_series_data = pd.read_csv('time_series_d.csv')

In [ ]:
business_data

In [ ]:
user_data

In [ ]:
time_series_data

In [ ]:
# Merge the time-series data with the business and user data
time_series_data = pd.merge(time_series_data, business_data, on='business_id', how='left')

In [ ]:
time_series_data = pd.merge(time_series_data, user_data, on='user_id', how='left')

In [ ]:
mm = pd.read_csv('merged_time_series.csv')

In [ ]:
time_series_data

In [ ]:
mm

In [ ]:
len(mm['business_id'].unique())

In [ ]:
mm = mm.drop(columns=['time','business_id',
                    'user_behavior','business_performance',
                   'value'])

In [ ]:
mm

In [ ]:
time_series_data = time_series_data.drop(columns=['suburb_name'])

In [ ]:
time_series_data

In [ ]:
data = pd.merge(time_series_data, mm, on='user_id', how='left')

In [ ]:
data.head(60)

In [ ]:
data2 = data.drop(columns=['user_id',
                           'business_id',
                          'business_description',
                           'business_location_lat','business_location_lon',
                          'user_location_lat','user_location_lon'])

In [ ]:
data2

In [ ]:
print(data2['user_behavior'].unique())

In [ ]:
data2.replace({'rated': 0, 'visited': 1}, inplace= True)

In [ ]:
print(data2['business_performance'].unique())

In [ ]:
data2.replace({'rating': 0, 'sales': 0.5}, inplace =True)


In [ ]:
print(data2['business_performance'].unique())

In [ ]:
data2['business_category_encoded'] = pd.factorize(data.business_category)[0]
data2['suburb_name_encoded'] = pd.factorize(data.suburb_name)[0]

In [ ]:
data2

In [ ]:
data0 = data2.sort_values(by='suburb_name')

In [ ]:
data0

In [ ]:
data3 = data0.drop(columns=['business_category'])

In [ ]:
data3

In [ ]:
scaling_columns = ['value','business_reviews','business_rating','suburb_name_encoded']
scale_factor = MinMaxScaler()
data3[scaling_columns] = scale_factor.fit_transform(data3[scaling_columns])

In [ ]:
data3

In [ ]:
x = data3.drop(columns=['business_category_encoded'])
y = data3['business_category_encoded']

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [ ]:
x1 = x_train.values.reshape(x_train.shape[0], x_train.shape[1], 1 )
y1 = y_train.values.reshape(-1,1)

In [ ]:
data3['time'] = pd.to_datetime(data3['time'])

In [ ]:
data3['time']

In [ ]:
grouped = data3.groupby(["Suburb"]).apply(lambda x: x.sort_values(["Timestamp"], ascending = True)).reset_index(drop=True)

In [ ]:
grouped

In [ ]:
print(x1.shape)
print(y1.shape)

In [ ]:
grouped

In [1]:
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(128, input_shape = (7, 1), return_sequences=True),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.LSTM(64, return_sequences =True),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.LSTM(32, return_sequences =False),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(208,activation='relu')
    tf.keras.layers.Dense(104, activation ='relu'),
    tf.keras.layers.Dense(52, activation ='softmax')
    ])

callbacks = [EarlyStopping(monitor='accuracy', patience = 15)]
optim = tf.keras.optimizers.Adam(learning_rate=0.001)

model.compile(optimizer = optim,
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

SyntaxError: invalid syntax (1184816431.py, line 14)

In [ ]:
for name, group in grouped:
    X_group = group['input']
    y_group = group['business_category_encoded']
    x1.append(X_group)
    y1.append(y_group)
    model.fit(X_group, y_group, 
              validation_split =0.11,
              batch_size =2048,
              epochs =5,
              callbacks =callbacks,
              verbose=1)

In [ ]:
result = data6.apply(lambda x: x)
print(result)

In [ ]:
data3

In [ ]:
print(x1.shape)
print(y1.shape)

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Loss vs epochs')
plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.legend(['Training', 'Validation'], loc='upper right')

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Accuracy vs. Epochs')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Training', 'Validation'], loc='lower right')
plt.show() 

In [ ]:
x2 = x_test.values.reshape(x_test.shape[0], x_test.shape[1], 1 )
y2 = y_test.values.reshape(-1,1)

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

pred = model.predict(x2)
predicted = np.argmax(pred, axis=1).reshape(-1,1)

matrix = confusion_matrix(y2, predicted)

In [ ]:
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

plt.figure(figsize=(30,30))
sns.heatmap(matrix, annot=True, fmt='g', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

In [ ]:
path = '/home/algoryc/Projects/Untitled Folder/weights/saving weights'

checkpointing = ModelCheckpoint(filepath=path,
                                save_weights_only=True,
                                save_freq='epoch',
                                monitor='accuracy',
                                save_best_only=True,
                                mode='max',
                                verbose=1)


In [ ]:
plot_model(model, to_file='lstm_model.png', show_shapes=False, show_layer_names=False)